### Grouping and Summarzing Data in Spark

In this notebook we demonstrate how to summarize and group data in a Spark DataFrame. It is based on material supplied by Cloudera under their Cloudera Academic Partner program and *Spark: The Definitive Guide* book by Bill Chambers and Matei Zaharia. 

Joins are computationally expensive. Therefore we may keep the results of such joins on the cluster for further analysis to save us from having to recreate it often.

Topics
- Summarizing data
- Grouping data
- Cross-tabulations
- Pivoting

In [0]:
# Load the joined rides data (combines the original rides, drivers, and riders data)
rides = spark.read.parquet("/mnt/cis442f-data/duocar/joined/")

#Keep it in memory to speed up the following paragraphs. Spark is also pretty good at keeping the right data in memory
rides.persist()

Out[15]: DataFrame[ride_id: string, rider_id: string, driver_id: string, date_time: timestamp, utc_offset: int, service: string, origin_lat: decimal(9,6), origin_lon: decimal(9,6), dest_lat: decimal(9,6), dest_lon: decimal(9,6), distance: int, duration: int, cancelled: boolean, star_rating: int, driver_birth_date: date, driver_start_date: date, driver_first_name: string, driver_last_name: string, driver_sex: string, driver_ethnicity: string, driver_student: boolean, driver_home_block: string, driver_home_lat: decimal(9,6), driver_home_lon: decimal(9,6), vehicle_make: string, vehicle_model: string, vehicle_year: int, vehicle_color: string, vehicle_grand: boolean, vehicle_noir: boolean, vehicle_elite: boolean, driver_rides: int, driver_stars: int, rider_birth_date: date, rider_start_date: date, rider_first_name: string, rider_last_name: string, rider_sex: string, rider_ethnicity: string, rider_student: boolean, rider_home_block: string, rider_home_lat: decimal(9,6), rider_home_lon: decimal(9,6), rider_work_lat: decimal(9,6), rider_work_lon: decimal(9,6), review: string]

#### Summarizing Data with Aggregation Functions

Spark provides a number of summarization (aggregate) functions.  We have already seen the `describe` method:

In [0]:
# We have already seen the `describe` method
rides.describe("distance").show()

+-------+-----------------+
summary| distance|
+-------+-----------------+
 count| 45841|
 mean|8904.418948103226|
 stddev|8797.103315891005|
 min| 304|
 max| 92204|
+-------+-----------------+

Use the `count`, `countDistinct`, and `approx_count_distinct` functions to get more refined counts:

**Note:** 
- The `count` function returns the number of rows with non-null values.
- That `count("*")` counts rows with null values
- Use `count(lit(1))` rather than `count(1)` as an alternative to `count("*")` but don't forget you have to import `lit`
- `countDistinct` is computationally expensive (i.e. can take a long time for big data). `approx_count_distinct` arrives at a pretty good estimate much more quickly (see https://databricks.com/blog/2016/05/19/approximate-algorithms-in-apache-spark-hyperloglog-and-quantiles.html)

In [0]:
# Use the `count`, `countDistinct`, and `approx_count_distinct` 
# functions to get more refined counts
# Note that count("*") counts rows with null values

from pyspark.sql.functions import count, mean, countDistinct, approx_count_distinct, lit
rides.withColumnRenamed("distance", "dist") \
  .select(count("*"), count("dist"), \
   countDistinct("dist"), approx_count_distinct("dist")).show()

+--------+-----------+--------------------+---------------------------+
count(1)|count(dist)|count(DISTINCT dist)|approx_count_distinct(dist)|
+--------+-----------+--------------------+---------------------------+
 48775| 45841| 14310| 14486|
+--------+-----------+--------------------+---------------------------+

In [0]:
# Use the `agg` method to achieve the same results

rides.withColumnRenamed("distance", "dist") \
  .agg(count(lit(1)), count("dist"), countDistinct("dist"), \
   approx_count_distinct("dist")).show()

+--------+-----------+-----------+---------------------------+
count(1)|count(dist)|count(dist)|approx_count_distinct(dist)|
+--------+-----------+-----------+---------------------------+
 48775| 45841| 14310| 14486|
+--------+-----------+-----------+---------------------------+

In [0]:
# You can `collect` a result if you want to use it programatically

avg_dist = rides.agg(mean("distance")).collect()
print(avg_dist)
print (avg_dist[0][0]) 

[Row(avg(distance)=8904.418948103226)]
8904.418948103226

In [0]:
# Use the `sum` and `sumDistinct` functions to compute a column sum
# Not sure what to use `sumDistinct` for though

from pyspark.sql.functions import sum, sumDistinct
rides.agg(sum("distance"), sumDistinct("distance")).show()

+-------------+----------------------+
sum(distance)|sum(DISTINCT distance)|
+-------------+----------------------+
 408187469| 162218471|
+-------------+----------------------+

Spark provides a number of summary statistics

**Note:** 
- `mean` is an alias for `avg`
- `stddev` is an alias for the sample standard deviation `stddev_samp`
- `variance` is an alias for the sample variance `var_samp`
- Population standard deviation and population variance are available via `stddev_pop` and `var_pop`, respectively
- Following examples use `format_number` function and the `alias` method to produce more easily read results

In [0]:
# Spark provides a number of summary statistics

from pyspark.sql.functions import mean, stddev, variance, skewness, kurtosis, format_number
rides.agg(format_number(mean("distance"),2).alias("average"), \
    format_number(stddev("distance"),2).alias("std_dev"), \
    format_number(variance("distance"),2).alias("variance"), \
    format_number(skewness("distance"),3).alias("skewness"), \
    format_number(kurtosis("distance"),3).alias("kurtosis")).show()

+--------+--------+-------------+--------+--------+
 average| std_dev| variance|skewness|kurtosis|
+--------+--------+-------------+--------+--------+
8,904.42|8,797.10|77,389,026.75| 3.544| 16.891|
+--------+--------+-------------+--------+--------+

In [0]:
# Use the `min` and `max` functions to compute the minimum and maximum, respectively

from pyspark.sql.functions import min, max
rides.agg(min("distance"), max("distance")).show() 

+-------------+-------------+
min(distance)|max(distance)|
+-------------+-------------+
 304| 92204|
+-------------+-------------+

In [0]:
# Use the `first` and `last` functions to compute the first and last values, respectively
# This is based on the rows in the DataFrame, not on values in the DataFrame

from pyspark.sql.functions import first, last
rides.agg(first("distance", ignorenulls=False), last("distance", ignorenulls=False)).show()

+---------------+--------------+
first(distance)|last(distance)|
+---------------+--------------+
 10123| 4605|
+---------------+--------------+

In [0]:
# Use the `corr`, `covar_samp`, or `covar_pop` method to measure the relationship between two columns
from pyspark.sql.functions import corr, covar_samp, covar_pop
rides\
  .withColumnRenamed("distance", "dist").withColumnRenamed("duration", "dur") \
  .groupBy("rider_student")\
  .agg(format_number(corr("dist", "dur"),6).alias("corr(dist, dur)"), \
       format_number(covar_samp("dist", "dur"),2).alias("covar_samp(dist, dur)"), \
       format_number(covar_pop("dist", "dur"),2).alias("covar_pop(dist, dur)"))\
  .show()

+-------------+---------------+---------------------+--------------------+
rider_student|corr(dist, dur)|covar_samp(dist, dur)|covar_pop(dist, dur)|
+-------------+---------------+---------------------+--------------------+
 true| 0.864519| 830,183.90| 830,037.69|
 false| 0.935451| 3,211,644.78| 3,211,564.81|
+-------------+---------------+---------------------+--------------------+

#### Grouping 
Use the `agg` method with the `groupBy` (or `groupby`) method to refine your analysis

In [0]:
# Use the `agg` method with the `groupBy` method to refine your analysis
rides\
  .groupBy("rider_student")\
  .agg(count("*"), count("distance").alias("count(dist)"), \
       format_number(mean("distance"),2).alias("avg(dist)"), \
       format_number(stddev("distance"),2).alias("stddev(dist)"))\
  .show()

+-------------+--------+-----------+---------+------------+
rider_student|count(1)|count(dist)|avg(dist)|stddev(dist)|
+-------------+--------+-----------+---------+------------+
 true| 5811| 5678| 5,747.72| 4,069.77|
 false| 42964| 40163| 9,350.69| 9,185.91|
+-------------+--------+-----------+---------+------------+

In [0]:
# You can use more than one groupby column
rides\
  .groupBy("rider_student", "service")\
  .agg(count("*"), count("distance").alias("count(dist)"), \
       format_number(mean("distance"),2).alias("avg(dist)"), \
       format_number(stddev("distance"),2).alias("stddev(dist)"))\
  .orderBy("rider_student", "service")\
  .show()

+-------------+-------+--------+-----------+---------+------------+
rider_student|service|count(1)|count(dist)|avg(dist)|stddev(dist)|
+-------------+-------+--------+-----------+---------+------------+
 false| Car| 29037| 27497| 9,527.00| 9,585.50|
 false| Elite| 1349| 1199| 8,455.31| 7,405.14|
 false| Grand| 4667| 4271| 8,729.13| 8,229.34|
 false| Noir| 7911| 7196| 9,195.12| 8,369.64|
 true| Car| 2349| 2342| 5,687.88| 4,023.51|
 true| Elite| 760| 713| 5,690.90| 3,881.54|
 true| Grand| 1530| 1498| 5,949.99| 4,339.96|
 true| Noir| 1172| 1125| 5,638.99| 3,902.93|
+-------------+-------+--------+-----------+---------+------------+

#### `rollup` and `cube` methods

Use the `rollup` method (instead of `groupBy`) to get partial subtotals. Where you see the `null` values is where you can find the partial and grand totals totals

**Note** that in the example the partial subtotals are given for each category in "rider_student" (i.e. the first parameter passed to the `rollup` method)

In [0]:
# Use the `rollup` method to get partial subtotals
# Where you see the `null` values is where you can find the partial and grand totals totals
#
# This is the grand total for all the riders
# +-------------+-------+--------+-----------+---------+------------+
# |rider_student|service|count(1)|count(dist)|avg(dist)|stddev(dist)|
# +-------------+-------+--------+-----------+---------+------------+
#           null|   null|   48775|      45841|    8,904|       8,797|
# This is the subtotal for non-student rides across services
# |        false|   null|   42964|      40163|    9,350|       9,185|
#
# This is the subtotal for student rides across services
# |         true|   null|    5811|       5678|    5,747|       4,069|

rides\
  .rollup("rider_student", "service")\
  .agg(count("*"), count("distance").alias("count(dist)"), \
       format_number(mean("distance"),0).alias("avg(dist)"), \
       format_number(stddev("distance"),0).alias("stddev(dist)"))\
  .orderBy("rider_student", "service")\
  .show()

+-------------+-------+--------+-----------+---------+------------+
rider_student|service|count(1)|count(dist)|avg(dist)|stddev(dist)|
+-------------+-------+--------+-----------+---------+------------+
 null| null| 48775| 45841| 8,904| 8,797|
 false| null| 42964| 40163| 9,351| 9,186|
 false| Car| 29037| 27497| 9,527| 9,586|
 false| Elite| 1349| 1199| 8,455| 7,405|
 false| Grand| 4667| 4271| 8,729| 8,229|
 false| Noir| 7911| 7196| 9,195| 8,370|
 true| null| 5811| 5678| 5,748| 4,070|
 true| Car| 2349| 2342| 5,688| 4,024|
 true| Elite| 760| 713| 5,691| 3,882|
 true| Grand| 1530| 1498| 5,950| 4,340|
 true| Noir| 1172| 1125| 5,639| 3,903|
+-------------+-------+--------+-----------+---------+------------+

In [0]:
# Consider the following example
# It is difficult to tell which `null` refers to `null` values in the data 
# and which indicates a grouping for aggregation

rides.rollup("rider_sex").agg(count("*")).orderBy("rider_sex").show() 

+---------+--------+
rider_sex|count(1)|
+---------+--------+
 null| 2250|
 null| 48775|
 female| 20091|
 male| 26434|
+---------+--------+

Which `null` is which? We next use the `grouping` function to distinguish between the null values (higher values indicate higher levels of aggregation).

In this case the 48775 is the total and 2250 isn the number of rows for which we are missing data on sex.

In [0]:
from pyspark.sql.functions import grouping
rides.rollup("rider_sex").agg(grouping("rider_sex"), count("*")).orderBy("rider_sex").show()

+---------+-------------------+--------+
rider_sex|grouping(rider_sex)|count(1)|
+---------+-------------------+--------+
 null| 0| 2250|
 null| 1| 48775|
 female| 0| 20091|
 male| 0| 26434|
+---------+-------------------+--------+

In [0]:
# This version adds a little logic to interpret the output of the grouping function

from pyspark.sql.functions import coalesce, col, lit, when, desc
rides \
    .rollup("rider_sex") \
    .agg(grouping("rider_sex"), count("*").alias("Count")) \
    .withColumn("rider_sex", when(col("grouping(rider_sex)") != 0, "Total").otherwise(col("rider_sex"))) \
    .orderBy(desc("Count")) \
    .drop("grouping(rider_sex)") \
    .show()

+---------+-----+
rider_sex|Count|
+---------+-----+
 Total|48775|
 male|26434|
 female|20091|
 null| 2250|
+---------+-----+

In [0]:
# Use the `cube` method to get all subtotals
rides\
  .cube("rider_student", "service")\
  .agg(count("*"), count("distance").alias("count(dist)"), \
       format_number(mean("distance"),0).alias("avg(dist)"), \
       format_number(stddev("distance"),0).alias("stddev(dist)"))\
  .orderBy("rider_student", "service")\
  .show()


+-------------+-------+--------+-----------+---------+------------+
rider_student|service|count(1)|count(dist)|avg(dist)|stddev(dist)|
+-------------+-------+--------+-----------+---------+------------+
 null| null| 48775| 45841| 8,904| 8,797|
 null| Car| 31386| 29839| 9,226| 9,328|
 null| Elite| 2109| 1912| 7,424| 6,464|
 null| Grand| 6197| 5769| 8,007| 7,517|
 null| Noir| 9083| 8321| 8,714| 8,007|
 false| null| 42964| 40163| 9,351| 9,186|
 false| Car| 29037| 27497| 9,527| 9,586|
 false| Elite| 1349| 1199| 8,455| 7,405|
 false| Grand| 4667| 4271| 8,729| 8,229|
 false| Noir| 7911| 7196| 9,195| 8,370|
 true| null| 5811| 5678| 5,748| 4,070|
 true| Car| 2349| 2342| 5,688| 4,024|
 true| Elite| 760| 713| 5,691| 3,882|
 true| Grand| 1530| 1498| 5,950| 4,340|
 true| Noir| 1172| 1125| 5,639| 3,903|
+-------------+-------+--------+-----------+---------+------------+

In [0]:
# Use the `grouping_id` function to distinguish grouping levels
# grouping_id 3: The highest level of aggregation (across "rider_student" and "service")
# grouping_id 2: Aggregations for "service" across "rider_student" categories (the first parameter)
# grouping_id 1: Aggregations for "rider_student" category across "service" categories
# grouping_id 0: Aggretations for each combinatin of "rider_student" and "service" category


from pyspark.sql.functions import grouping_id
rides\
  .cube("rider_student", "service")\
  .agg(grouping_id("rider_student", "service").alias("grouping_id"), \
       count("*"), count("distance").alias("count(dist)"), \
       format_number(mean("distance"),0).alias("avg(dist)"), \
       format_number(stddev("distance"),0).alias("stddev(dist)"))\
  .orderBy("rider_student", "service")\
  .withColumnRenamed("rider_student","student")\
  .show()


+-------+-------+-----------+--------+-----------+---------+------------+
student|service|grouping_id|count(1)|count(dist)|avg(dist)|stddev(dist)|
+-------+-------+-----------+--------+-----------+---------+------------+
 null| null| 3| 48775| 45841| 8,904| 8,797|
 null| Car| 2| 31386| 29839| 9,226| 9,328|
 null| Elite| 2| 2109| 1912| 7,424| 6,464|
 null| Grand| 2| 6197| 5769| 8,007| 7,517|
 null| Noir| 2| 9083| 8321| 8,714| 8,007|
 false| null| 1| 42964| 40163| 9,351| 9,186|
 false| Car| 0| 29037| 27497| 9,527| 9,586|
 false| Elite| 0| 1349| 1199| 8,455| 7,405|
 false| Grand| 0| 4667| 4271| 8,729| 8,229|
 false| Noir| 0| 7911| 7196| 9,195| 8,370|
 true| null| 1| 5811| 5678| 5,748| 4,070|
 true| Car| 0| 2349| 2342| 5,688| 4,024|
 true| Elite| 0| 760| 713| 5,691| 3,882|
 true| Grand| 0| 1530| 1498| 5,950| 4,340|
 true| Noir| 0| 1172| 1125| 5,639| 3,903|
+-------+-------+-----------+--------+-----------+---------+------------+

#### Cross-tabulations

In [0]:
# The following use case is common
rides.groupby("rider_student", "service").count().orderBy("rider_student", "service").show()

+-------------+-------+-----+
rider_student|service|count|
+-------------+-------+-----+
 false| Car|29037|
 false| Elite| 1349|
 false| Grand| 4667|
 false| Noir| 7911|
 true| Car| 2349|
 true| Elite| 760|
 true| Grand| 1530|
 true| Noir| 1172|
+-------------+-------+-----+

In [0]:
# The `crosstab` method is a more direct way to get this result:
rides.crosstab("rider_student", "service").show()

+---------------------+-----+-----+-----+----+
rider_student_service| Car|Elite|Grand|Noir|
+---------------------+-----+-----+-----+----+
 false|29037| 1349| 4667|7911|
 true| 2349| 760| 1530|1172|
+---------------------+-----+-----+-----+----+

#### Pivoting

In [0]:
# We can also use the `pivot` method to produce a cross-tabulation:
rides.groupby("rider_student").pivot("service").count().show()

+-------------+-----+-----+-----+----+
rider_student| Car|Elite|Grand|Noir|
+-------------+-----+-----+-----+----+
 true| 2349| 760| 1530|1172|
 false|29037| 1349| 4667|7911|
+-------------+-----+-----+-----+----+

In [0]:
# We can also do other aggregations 
# Note: second version uses the `agg` method with the mean and round 
# functions. This allows more fine control of the output format
from pyspark.sql.functions import round
rides.groupby("rider_student").pivot("service").mean("distance").show()
rides.groupby("rider_student").pivot("service").agg(round(mean("distance"),2)).show()

+-------------+-----------------+------------------+-----------------+-----------------+
rider_student| Car| Elite| Grand| Noir|
+-------------+-----------------+------------------+-----------------+-----------------+
 true|5687.875320239112|5690.9004207573635|5949.985981308411|5638.986666666667|
 false| 9526.99639960723| 8455.314428690575|8729.128073050808|9195.117565314064|
+-------------+-----------------+------------------+-----------------+-----------------+

+-------------+-------+-------+-------+-------+
rider_student| Car| Elite| Grand| Noir|
+-------------+-------+-------+-------+-------+
 true|5687.88| 5690.9|5949.99|5638.99|
 false| 9527.0|8455.31|8729.13|9195.12|
+-------------+-------+-------+-------+-------+

In [0]:
# You can explicity choose the values that are pivoted to columns
rides.groupby("rider_student").pivot("service", ["Grand", "Car"]).agg(mean("distance")).show()

+-------------+-----------------+-----------------+
rider_student| Grand| Car|
+-------------+-----------------+-----------------+
 true|5949.985981308411|5687.875320239112|
 false|8729.128073050808| 9526.99639960723|
+-------------+-----------------+-----------------+

In [0]:
# Additional aggregation functions produce additional columns
rides.groupby("rider_student").pivot("service", ["Car"]).agg(count("distance"), mean("distance")).show()

+-------------+-------------------+---------------------------------+
rider_student|Car_count(distance)|Car_avg(CAST(distance AS BIGINT))|
+-------------+-------------------+---------------------------------+
 true| 2342| 5687.875320239112|
 false| 27497| 9526.99639960723|
+-------------+-------------------+---------------------------------+

In [0]:
# Free up memory
# Unpersist DataFrame
rides.unpersist()

# Note that when running a larger piece of analysis Spark does a pretty good job of figuring out what data to keep in memory

Out[28]: DataFrame[ride_id: string, rider_id: string, driver_id: string, date_time: timestamp, utc_offset: int, service: string, origin_lat: decimal(9,6), origin_lon: decimal(9,6), dest_lat: decimal(9,6), dest_lon: decimal(9,6), distance: int, duration: int, cancelled: boolean, star_rating: int, driver_birth_date: date, driver_start_date: date, driver_first_name: string, driver_last_name: string, driver_sex: string, driver_ethnicity: string, driver_student: boolean, driver_home_block: string, driver_home_lat: decimal(9,6), driver_home_lon: decimal(9,6), vehicle_make: string, vehicle_model: string, vehicle_year: int, vehicle_color: string, vehicle_grand: boolean, vehicle_noir: boolean, vehicle_elite: boolean, driver_rides: int, driver_stars: int, rider_birth_date: date, rider_start_date: date, rider_first_name: string, rider_last_name: string, rider_sex: string, rider_ethnicity: string, rider_student: boolean, rider_home_block: string, rider_home_lat: decimal(9,6), rider_home_lon: decimal(9,6), rider_work_lat: decimal(9,6), rider_work_lon: decimal(9,6), review: string]

###Hands On

![Hands-on](https://cis442f-open-data.s3.amazonaws.com/pictures/hands.png "Hands-on")


#### Exercises

1. Who are DuoCar's top 10 riders in terms of number of rides?

2. Who are DuoCar's top 10 drivers in terms of total distance driven?

3. Does star rating depend on vehicle make?

4. How do star ratings depend on rider sex and student status?

5.  Explore the distance data. Do nulls correspond to canceled trips? 

6. Investigate student use of duocar (see hint below)
   - a. When do they take rides by day of week and by hour of day (include at least two visualization - see notebook 7b for ideas on how)?
   - b. Does the use of different types of car service (e.g. Noir) change by time of day or day of week?
   - c. Does usage pattern differ by sex? 

**Hint** You use this to get local time: `expr('from_utc_timestamp(date_time, "America/North_Dakota/Center")').alias("local_time"))`

**References you might need**

See the [pyspark function documentation](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#functions) to find out about the aggregation (and other) functions.

In [0]:
|
